In [1]:
# To be run after Adult-Data-Prep
# Import Data handling/display libraries
import sys
import pandas as pd
import numpy  as np 
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble      import RandomForestClassifier
from sklearn.linear_model  import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics       import accuracy_score
from sklearn.metrics       import balanced_accuracy_score
from sklearn.metrics       import auc, roc_auc_score, roc_curve, classification_report, confusion_matrix
from IPython.display       import Markdown, display
# Import IBM's AI Fairness tooolbox
from aif360.datasets       import BinaryLabelDataset
from aif360.metrics        import BinaryLabelDatasetMetric
from aif360.metrics        import ClassificationMetric
from aif360.metrics.utils  import compute_boolean_conditioning_vector
#from aif360.algorithms.preprocessing.optim_preproc_helpers.data_preproc_functions import load_preproc_data_adult, load_preproc_data_compas, load_preproc_data_german
from aif360.algorithms.inprocessing.adversarial_debiasing import AdversarialDebiasing
%matplotlib inline
# Warnings will be used to silence various model warnings for tidier output
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Read the cleaned Taiwan-Credit-Card-Cleaned-Marriage dataset
Taiwan_df = pd.read_csv('./input/Taiwan-Credit-Card-Cleaned-Gender.csv')

In [3]:
# Set privileged (1)/ unprivileged (0)/ favourable (1) / unfavourable values (0)
protected_attr      = 'GENDER'
priv_grp            = 1  # Males 
unpriv_grp          = 0  # Females  
lab                 = 'DEFAULT'
fav_label           = 1 # Will not default next month
unfav_label         = 0 # Will default next month
privileged_groups   = [{protected_attr: priv_grp}]   # Males
unprivileged_groups = [{protected_attr: unpriv_grp}] # Females

In [4]:
# Create the traoning and test splits
X = Taiwan_df.drop(lab,axis=1)
y = Taiwan_df[lab]

In [5]:
# Create a Binary Label Dataset to use with AIF360 APIs
Taiwan_bld = BinaryLabelDataset(df=pd.concat((X, y), axis=1),
                                label_names=[lab], protected_attribute_names=[protected_attr],
                                favorable_label=fav_label, unfavorable_label=unfav_label)

In [6]:
# Create train and test datasets
Taiwan_train_bld, Taiwan_test_bld = Taiwan_bld.split([0.8], shuffle=True)

In [7]:
#Scale the binary labelled datasets
min_max_scaler = MinMaxScaler()
Taiwan_train_bld.features = min_max_scaler.fit_transform(Taiwan_train_bld.features)
Taiwan_test_bld.features  = min_max_scaler.fit_transform(Taiwan_test_bld.features)

In [8]:
# Convert the train and test datasets to dataframes
Taiwan_train_df, d = Taiwan_train_bld.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Taiwan_test_df,  d = Taiwan_test_bld.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [9]:
# Determine the baseline model accuracy for Logistic Regression and Random Forest Classifiers
X_train = Taiwan_train_df.drop(lab,axis=1)
y_train = Taiwan_train_df[lab]
X_test  = Taiwan_test_df.drop(lab,axis=1)
y_test  = Taiwan_test_df[lab]
BiasedLogModel = LogisticRegression(random_state=101)
BiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
BiasedLogModel.fit(X_train, y_train) 
BiasedRfcModel.fit(X_train, y_train)
BiasedLogPredictions = BiasedLogModel.predict(X_test)
BiasedRfcPredictions = BiasedRfcModel.predict(X_test)
print(f"Biased Logistic regression validation accuracy: {BiasedLogModel.score(X_test, y_test)}")
print(f"Biased Random Forest       validation accuracy: {BiasedRfcModel.score(X_test, y_test)}")
print('')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_test, BiasedLogPredictions)}")
print(f"Biased Random forest balanced accuracy        : {balanced_accuracy_score(y_test, BiasedRfcPredictions)}")

Biased Logistic regression validation accuracy: 0.8056666666666666
Biased Random Forest       validation accuracy: 0.8086666666666666

Biased Logistic regression balanced accuracy  : 0.5881280319977173
Biased Random forest balanced accuracy        : 0.6079897256686702


In [10]:
print('Training Before - DEFAULT value counts:')
print(Taiwan_train_df.DEFAULT.value_counts())
print('Training Before - GENDER value counts:')
print(Taiwan_train_df.GENDER.value_counts())

Training Before - DEFAULT value counts:
1.0    18698
0.0     5302
Name: DEFAULT, dtype: int64
Training Before - GENDER value counts:
0.0    14549
1.0     9451
Name: GENDER, dtype: int64


In [11]:
print('Test Before - DEFAULT value counts:')
print(Taiwan_test_df.DEFAULT.value_counts())
print('Test Before - GENDER value counts:')
print(Taiwan_test_df.GENDER.value_counts())

Test Before - DEFAULT value counts:
1.0    4666
0.0    1334
Name: DEFAULT, dtype: int64
Test Before - GENDER value counts:
0.0    3563
1.0    2437
Name: GENDER, dtype: int64


In [12]:
# Create the binary label dataset metric class for the training dataset
metric_train_bld = BinaryLabelDatasetMetric(Taiwan_train_bld, 
                                            unprivileged_groups=unprivileged_groups,
                                            privileged_groups=privileged_groups)
metric_test_bld = BinaryLabelDatasetMetric(Taiwan_test_bld, 
                                           unprivileged_groups=unprivileged_groups,
                                           privileged_groups=privileged_groups)

In [13]:
display(Markdown("#### Orig training dataset"))
print('Number of instances           :', metric_train_bld.num_instances())
print('Base Rate                     :', metric_train_bld.base_rate())
print('Consistency                   :', metric_train_bld.consistency())
print('Disparate Impact              :', metric_train_bld.disparate_impact())
print('Mean Difference               :', metric_train_bld.mean_difference())
print('Statistical Parity Difference :', metric_train_bld.statistical_parity_difference()) 
display(Markdown("#### Orig test dataset"))
print('Number of instances           :', metric_test_bld.num_instances())
print('Base Rate                     :', metric_test_bld.base_rate())
print('Consistency                   :', metric_test_bld.consistency())
print('Disparate Impact              :', metric_test_bld.disparate_impact())
print('Mean Difference               :', metric_test_bld.mean_difference())
print('Statistical Parity Difference :', metric_test_bld.statistical_parity_difference()) 

#### Orig training dataset

Number of instances           : 24000.0
Base Rate                     : 0.7790833333333333
Consistency                   : [0.783825]
Disparate Impact              : 1.0401880843019193
Mean Difference               : 0.03056522589802102
Statistical Parity Difference : 0.03056522589802102


#### Orig test dataset

Number of instances           : 6000.0
Base Rate                     : 0.7776666666666666
Consistency                   : [0.78366667]
Disparate Impact              : 1.062836723786911
Mean Difference               : 0.04710820449679376
Statistical Parity Difference : 0.04710820449679376


In [14]:
# Load post-processing algorithm that equalizes the odds
# Learn parameters with debias set to FALSE!
sess = tf.Session()
biased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=False,
                          sess=sess)

In [15]:
biased_model.fit(Taiwan_train_bld)

W0914 00:07:20.129674  9496 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:138: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0914 00:07:20.131673  9496 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:142: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0914 00:07:20.144665  9496 deprecation_wrapper.py:119] From c:\users\befaria\appdata\local\continuum\miniconda3\lib\site-packages\aif360\algorithms\inprocessing\adversarial_debiasing.py:87: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0914 00:07:22.624666  9496 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https:

epoch 0; iter: 0; batch classifier loss: 0.657834
epoch 1; iter: 0; batch classifier loss: 0.535646
epoch 2; iter: 0; batch classifier loss: 0.421517
epoch 3; iter: 0; batch classifier loss: 0.486037
epoch 4; iter: 0; batch classifier loss: 0.410988
epoch 5; iter: 0; batch classifier loss: 0.377697
epoch 6; iter: 0; batch classifier loss: 0.429664
epoch 7; iter: 0; batch classifier loss: 0.365086
epoch 8; iter: 0; batch classifier loss: 0.443991
epoch 9; iter: 0; batch classifier loss: 0.456086
epoch 10; iter: 0; batch classifier loss: 0.493173
epoch 11; iter: 0; batch classifier loss: 0.499553
epoch 12; iter: 0; batch classifier loss: 0.405097
epoch 13; iter: 0; batch classifier loss: 0.436264
epoch 14; iter: 0; batch classifier loss: 0.405587
epoch 15; iter: 0; batch classifier loss: 0.459541
epoch 16; iter: 0; batch classifier loss: 0.426845
epoch 17; iter: 0; batch classifier loss: 0.449122
epoch 18; iter: 0; batch classifier loss: 0.475301
epoch 19; iter: 0; batch classifier loss:

In [16]:
# Apply the plain model to test data
Taiwan_biased_train = biased_model.predict(Taiwan_train_bld)
Taiwan_biased_test  = biased_model.predict(Taiwan_test_bld)
# And convert them to dataframes
Taiwan_train_df, d = Taiwan_biased_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Taiwan_test_df, d  = Taiwan_biased_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [17]:
# Test whether it is possible to predict the Sensitive Variable from the whole original dataset
X_se_train = Taiwan_train_df.drop(protected_attr,axis=1)
y_se_train = Taiwan_train_df[protected_attr]
X_se_test  = Taiwan_test_df.drop(protected_attr,axis=1)
y_se_test  = Taiwan_test_df[protected_attr]

Se_BiasedLogModel = LogisticRegression(random_state=101)
Se_BiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
Se_BiasedLogModel.fit(X_se_train, y_se_train) 
Se_BiasedRfcModel.fit(X_se_train, y_se_train) 
yseLog_pred =  Se_BiasedLogModel.predict(X_se_test)
yseRfc_pred =  Se_BiasedRfcModel.predict(X_se_test)
# Now test whether we can predict Gender from the test dataset
print('Model Accuracy for predicting the Sensitive Variable before bias transformation:')
print(f"Biased Logistic regression validation accuracy: {Se_BiasedLogModel.score(X_se_test, y_se_test)}")
print(f"Biased Random Forest       validation accuracy: {Se_BiasedRfcModel.score(X_se_test, y_se_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_se_test, yseLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_se_test, yseRfc_pred)}")

Model Accuracy for predicting the Sensitive Variable before bias transformation:
Biased Logistic regression validation accuracy: 0.5946666666666667
Biased Random Forest       validation accuracy: 0.5985

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.5178840199925578
Biased Random Forest       balanced accuracy  : 0.50652283747461


In [18]:
# Metrics for the dataset from plain model (without debiasing)
display(Markdown("#### Plain model - without debiasing - dataset metrics"))
metric_biased_train = BinaryLabelDatasetMetric(Taiwan_biased_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

metric_biased_test = BinaryLabelDatasetMetric(Taiwan_biased_test, 
                            unprivileged_groups=unprivileged_groups,
                            privileged_groups=privileged_groups)
display(Markdown("#### Biased training dataset"))
print('Number of instances           :', metric_biased_train.num_instances())
print('Base Rate                     :', metric_biased_train.base_rate())
print('Consistency                   :', metric_biased_train.consistency())
print('Disparate Impact              :', metric_biased_train.disparate_impact())
print('Mean Difference               :', metric_biased_train.mean_difference())
print('Statistical Parity Difference :', metric_biased_train.statistical_parity_difference()) 
display(Markdown("#### Biased test dataset"))
print('Number of instances           :', metric_biased_test.num_instances())
print('Base Rate                     :', metric_biased_test.base_rate())
print('Consistency                   :', metric_biased_test.consistency())
print('Disparate Impact              :', metric_biased_test.disparate_impact())
print('Mean Difference               :', metric_biased_test.mean_difference())
print('Statistical Parity Difference :', metric_biased_test.statistical_parity_difference()) 

#### Plain model - without debiasing - dataset metrics

#### Biased training dataset

Number of instances           : 24000.0
Base Rate                     : 0.89075
Consistency                   : [0.97286667]
Disparate Impact              : 1.0255654345893719
Mean Difference               : 0.022424870674626285
Statistical Parity Difference : 0.022424870674626285


#### Biased test dataset

Number of instances           : 6000.0
Base Rate                     : 0.9348333333333333
Consistency                   : [0.9664]
Disparate Impact              : 1.0248949391387727
Mean Difference               : 0.022933581603013953
Statistical Parity Difference : 0.022933581603013953


In [19]:
# Create the Classification metric for train and test
classified_metric_biased_test = ClassificationMetric(Taiwan_train_bld, Taiwan_biased_train,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
classified_metric_biased_train = ClassificationMetric(Taiwan_test_bld, Taiwan_biased_test,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)

In [20]:
display(Markdown("#### Plain model - without debiasing - classification metrics"))

display(Markdown("#### Biased training dataset - CLassification Metrics"))
TPR_train = classified_metric_biased_train.true_positive_rate()
TNR_train = classified_metric_biased_train.true_negative_rate()
bal_acc_biased_train = 0.5*(TPR_train + TNR_train)
print('Classification accuracy         =', classified_metric_biased_train.accuracy())
print('Balanced classification accuracy=', bal_acc_biased_train)
print('Disparate impact                =', classified_metric_biased_train.disparate_impact())
print('Equal opportunity difference    =', classified_metric_biased_train.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_biased_train.average_odds_difference())
print('Theil_index                     =', classified_metric_biased_train.theil_index())

display(Markdown("#### Biased test dataset - CLassification Metrics"))
TPR_test = classified_metric_biased_test.true_positive_rate()
TNR_test = classified_metric_biased_test.true_negative_rate()
bal_acc_biased_test = 0.5*(TPR_test+TNR_test)
print('Classification accuracy         =', classified_metric_biased_test.accuracy())
print('Balanced classification accuracy=', bal_acc_biased_test)
print('Disparate impact                =', classified_metric_biased_test.disparate_impact())
print('Equal opportunity difference    =', classified_metric_biased_test.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_biased_test.average_odds_difference())
print('Theil_index                     =', classified_metric_biased_test.theil_index())

#### Plain model - without debiasing - classification metrics

#### Biased training dataset - CLassification Metrics

Classification accuracy         = 0.8065
Balanced classification accuracy= 0.5940169113899972
Disparate impact                = 1.0248949391387727
Equal opportunity difference    = 0.008384467421768127
Average odds difference         = 0.02088003372899838
Theil_index                     = 0.06407616216959693


#### Biased test dataset - CLassification Metrics

Classification accuracy         = 0.8218333333333333
Balanced classification accuracy= 0.6506713813910225
Disparate impact                = 1.0255654345893719
Equal opportunity difference    = 0.01225560798067249
Average odds difference         = 0.014407849363077097
Theil_index                     = 0.07485669280783086


In [21]:
sess.close()
tf.reset_default_graph()
sess = tf.Session()

In [22]:
unbiased_model = AdversarialDebiasing(privileged_groups = privileged_groups,
                          unprivileged_groups = unprivileged_groups,
                          scope_name='plain_classifier',
                          debias=True,
                          sess=sess)

In [23]:
unbiased_model.fit(Taiwan_train_bld)

epoch 0; iter: 0; batch classifier loss: 0.692761; batch adversarial loss: 0.710298
epoch 1; iter: 0; batch classifier loss: 0.457519; batch adversarial loss: 0.737755
epoch 2; iter: 0; batch classifier loss: 0.404200; batch adversarial loss: 0.695146
epoch 3; iter: 0; batch classifier loss: 0.455199; batch adversarial loss: 0.695964
epoch 4; iter: 0; batch classifier loss: 0.464165; batch adversarial loss: 0.675323
epoch 5; iter: 0; batch classifier loss: 0.420111; batch adversarial loss: 0.668298
epoch 6; iter: 0; batch classifier loss: 0.430076; batch adversarial loss: 0.676728
epoch 7; iter: 0; batch classifier loss: 0.383151; batch adversarial loss: 0.674743
epoch 8; iter: 0; batch classifier loss: 0.429564; batch adversarial loss: 0.677012
epoch 9; iter: 0; batch classifier loss: 0.440074; batch adversarial loss: 0.635771
epoch 10; iter: 0; batch classifier loss: 0.542047; batch adversarial loss: 0.656798
epoch 11; iter: 0; batch classifier loss: 0.424100; batch adversarial loss:

In [24]:
Taiwan_unbiased_train = unbiased_model.predict(Taiwan_train_bld)
Taiwan_unbiased_test  = unbiased_model.predict(Taiwan_test_bld)
# And convert these to dataframes..
Taiwan_train_df, d = Taiwan_unbiased_train.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)
Taiwan_test_df, d  = Taiwan_unbiased_test.convert_to_dataframe(de_dummy_code=False, sep='=', set_category=False)

In [25]:
# Metrics for the dataset from plain model (after unbiasing)
display(Markdown("#### Plain model - after unbiasing - dataset metrics"))
metric_unbiased_train = BinaryLabelDatasetMetric(Taiwan_unbiased_train, 
                             unprivileged_groups=unprivileged_groups,
                             privileged_groups=privileged_groups)

metric_unbiased_test = BinaryLabelDatasetMetric(Taiwan_unbiased_test, 
                            unprivileged_groups=unprivileged_groups,
                            privileged_groups=privileged_groups)
display(Markdown("#### Biased training dataset"))
print('Number of instances           :', metric_unbiased_train.num_instances())
print('Base Rate                     :', metric_unbiased_train.base_rate())
print('Consistency                   :', metric_unbiased_train.consistency())
print('Disparate Impact              :', metric_unbiased_train.disparate_impact())
print('Mean Difference               :', metric_unbiased_train.mean_difference())
print('Statistical Parity Difference :', metric_unbiased_train.statistical_parity_difference()) 
display(Markdown("#### Biased test dataset"))
print('Number of instances           :', metric_unbiased_test.num_instances())
print('Base Rate                     :', metric_unbiased_test.base_rate())
print('Consistency                   :', metric_unbiased_test.consistency())
print('Disparate Impact              :', metric_unbiased_test.disparate_impact())
print('Mean Difference               :', metric_unbiased_test.mean_difference())
print('Statistical Parity Difference :', metric_unbiased_test.statistical_parity_difference()) 

#### Plain model - after unbiasing - dataset metrics

#### Biased training dataset

Number of instances           : 24000.0
Base Rate                     : 0.8862916666666667
Consistency                   : [0.97418333]
Disparate Impact              : 1.0216160655965894
Mean Difference               : 0.018910340741995713
Statistical Parity Difference : 0.018910340741995713


#### Biased test dataset

Number of instances           : 6000.0
Base Rate                     : 0.9165
Consistency                   : [0.96363333]
Disparate Impact              : 1.0248736222391706
Mean Difference               : 0.022464851271405073
Statistical Parity Difference : 0.022464851271405073


In [26]:
# Create the Classification metric for train and test
classified_metric_unbiased_train = ClassificationMetric(Taiwan_train_bld, Taiwan_unbiased_train,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)
classified_metric_unbiased_test = ClassificationMetric(Taiwan_test_bld, Taiwan_unbiased_test,
                                       unprivileged_groups=unprivileged_groups,
                                       privileged_groups=privileged_groups)

In [27]:
display(Markdown("#### Plain model - after debiasing - classification metrics"))
display(Markdown("#### Unbiased training dataset - CLassification Metrics"))
TPR_train = classified_metric_unbiased_train.true_positive_rate()
TNR_train = classified_metric_unbiased_train.true_negative_rate()
bal_acc_unbiased_train = 0.5*(TPR_train + TNR_train)
print('Classification accuracy         =', classified_metric_unbiased_train.accuracy())
print('Balanced classification accuracy=', bal_acc_unbiased_train)
print('Disparate impact                =', classified_metric_unbiased_train.disparate_impact())
print('Equal opportunity difference    =', classified_metric_unbiased_train.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_unbiased_train.average_odds_difference())
print('Theil_index                     =', classified_metric_unbiased_train.theil_index())

display(Markdown("#### Unbiased test dataset - CLassification Metrics"))
TPR_test = classified_metric_unbiased_test.true_positive_rate()
TNR_test = classified_metric_unbiased_test.true_negative_rate()
bal_acc_unbiased_test = 0.5*(TPR_test+TNR_test)
print('Classification accuracy         =', classified_metric_unbiased_test.accuracy())
print('Balanced classification accuracy=', bal_acc_unbiased_test)
print('Disparate impact                =', classified_metric_unbiased_test.disparate_impact())
print('Equal opportunity difference    =', classified_metric_unbiased_test.equal_opportunity_difference())
print('Average odds difference         =', classified_metric_unbiased_test.average_odds_difference())
print('Theil_index                     =', classified_metric_unbiased_test.theil_index())

#### Plain model - after debiasing - classification metrics

#### Unbiased training dataset - CLassification Metrics

Classification accuracy         = 0.8217083333333334
Balanced classification accuracy= 0.6541044457398038
Disparate impact                = 1.0216160655965894
Equal opportunity difference    = 0.009511170030947369
Average odds difference         = 0.009491339401270227
Theil_index                     = 0.07689013172246865


#### Unbiased test dataset - CLassification Metrics

Classification accuracy         = 0.8098333333333333
Balanced classification accuracy= 0.6135576125353526
Disparate impact                = 1.0248736222391706
Equal opportunity difference    = 0.012324621499780308
Average odds difference         = 0.011186765056565295
Theil_index                     = 0.07024042650497817


In [28]:
# Determine the after-transformation model accuracy for Logistic Regression and Random Forest Classifiers
X_train = Taiwan_train_df.drop(lab,axis=1)
y_train = Taiwan_train_df[lab]
X_test  = Taiwan_test_df.drop(lab,axis=1)
y_test  = Taiwan_test_df[lab]

UnbiasedLogModel = LogisticRegression(random_state=101)
UnbiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
UnbiasedLogModel.fit(X_train, y_train) 
UnbiasedRfcModel.fit(X_train, y_train) 
UnbiasedLog_pred = UnbiasedLogModel.predict(X_test)
UnbiasedRfc_pred = UnbiasedRfcModel.predict(X_test)

# Now get Logistic Regression and Random Forest Clasisfication performancce of unbiased data
print('Model Accuracy for Log Reg and RFC after bias transformation:')
print(f"Biased Logistic regression validation accuracy: {UnbiasedLogModel.score(X_test, y_test)}")
print(f"Biased Random Forest       validation accuracy: {UnbiasedRfcModel.score(X_test, y_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_test, UnbiasedLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_test, UnbiasedRfc_pred)}")

Model Accuracy for Log Reg and RFC after bias transformation:
Biased Logistic regression validation accuracy: 0.9695
Biased Random Forest       validation accuracy: 0.9746666666666667

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.9706615138517292
Biased Random Forest       balanced accuracy  : 0.9136130357942054


In [29]:
# Finally test whether it is possible to predict the Sensitive Variable from the transformed dataset
X_se_train = Taiwan_train_df.drop(protected_attr,axis=1)
y_se_train = Taiwan_train_df[protected_attr]
X_se_test  = Taiwan_test_df.drop(protected_attr,axis=1)
y_se_test  = Taiwan_test_df[protected_attr]

Se_BiasedLogModel = LogisticRegression(random_state=101)
Se_BiasedRfcModel = RandomForestClassifier(n_estimators=100,max_depth=4,random_state=101)
Se_BiasedLogModel.fit(X_se_train, y_se_train) 
Se_BiasedRfcModel.fit(X_se_train, y_se_train) 
yseLog_pred =  Se_BiasedLogModel.predict(X_se_test)
yseRfc_pred =  Se_BiasedRfcModel.predict(X_se_test)

# Now test whether we can predict the protected_attr from the test dataset
print('Model Accuracy for predicting the Sensitive Variable after bias transformation:')
print(f"Biased Logistic regression validation accuracy: {Se_BiasedLogModel.score(X_se_test, y_se_test)}")
print(f"Biased Random Forest       validation accuracy: {Se_BiasedRfcModel.score(X_se_test, y_se_test)}")
print('')
print('Biased Balanced accuracy')
print(f"Biased Logistic regression balanced accuracy  : {balanced_accuracy_score(y_se_test, yseLog_pred)}")
print(f"Biased Random Forest       balanced accuracy  : {balanced_accuracy_score(y_se_test, yseRfc_pred)}")

Model Accuracy for predicting the Sensitive Variable after bias transformation:
Biased Logistic regression validation accuracy: 0.5945
Biased Random Forest       validation accuracy: 0.5996666666666667

Biased Balanced accuracy
Biased Logistic regression balanced accuracy  : 0.5178085279207226
Biased Random Forest       balanced accuracy  : 0.5084129032822755


In [30]:
print('Predicted Training After - DEFAULT value counts:')
print(Taiwan_train_df.DEFAULT.value_counts())
print('Predicted Training After - GENDER value counts:')
print(Taiwan_train_df.GENDER.value_counts())

Predicted Training After - DEFAULT value counts:
1.0    21271
0.0     2729
Name: DEFAULT, dtype: int64
Predicted Training After - GENDER value counts:
0.0    14549
1.0     9451
Name: GENDER, dtype: int64


In [31]:
print('Predicted Test After - DEFAULT value counts:')
print(Taiwan_test_df.DEFAULT.value_counts())
print('Predicted Test After - GENDER value counts:')
print(Taiwan_test_df.GENDER.value_counts())

Predicted Test After - DEFAULT value counts:
1.0    5499
0.0     501
Name: DEFAULT, dtype: int64
Predicted Test After - GENDER value counts:
0.0    3563
1.0    2437
Name: GENDER, dtype: int64
